# SUMMA workflow: make folder structure
Makes the initial folder structure for a given control file. All other files in the workflow will look for the file `control_active.txt` during their execution. This script:

1. Copies the specified control file into `control_active.txt`;
2. Prepares a folder structure using the settings in `control_active.txt`.
3. Creates a copy of itself to be stored in the new folder structure.

The destination folders are referred to as "domain folders".

In [37]:
# Specify the control file to use
sourceFile  = 'control_Bow_at_Banff_laptop.txt'

Do not change below this line.

In [38]:
# Modules
import os
from pathlib import Path
from shutil import copyfile
from datetime import datetime

#### Copy the control file into `control_active.txt`

In [39]:
# Easy access to control file folder
controlFolder = Path('../0_controlFiles')

In [40]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [41]:
copyfile( controlFolder/sourceFile, controlFolder/controlFile );

#### Create the main domain folder

In [42]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line:
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [43]:
# Find the path where the domain folders need to go
# Immediately store as a 'Path' to avoid issues with '/' and '\' on different operating systems
rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )

In [44]:
# Find the domain name
domainName = read_from_control(controlFolder/controlFile,'domain_name')

In [45]:
# Create the domain folder inside 'root'
domainFolder = 'domain_' + domainName
Path( rootPath / domainFolder ).mkdir(parents=True, exist_ok=True)

#### Make the shapefile folders

In [46]:
# Find the catchment shapefile folder in 'control_active'
catchmentShapeFolder = read_from_control(controlFolder/controlFile,'catchment_shp_path')
networkShapeFolder = read_from_control(controlFolder/controlFile,'river_network_path')

In [47]:
# Specify the default paths if required
if catchmentShapeFolder == 'default':
    catchmentShapeFolder = 'shapefiles/catchment'
if networkShapeFolder == 'default':
    networkShapeFolder = 'shapefiles/river_network'

In [48]:
# Try to make the shapefile folders; does nothing if the folder already exists
Path( rootPath / domainFolder / catchmentShapeFolder ).mkdir(parents=True, exist_ok=True)
Path( rootPath / domainFolder / networkShapeFolder ).mkdir(parents=True, exist_ok=True)

#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [49]:
# Create a log folder
logFolder = '_workflow_log'
Path( rootPath / domainFolder / logFolder ).mkdir(parents=True, exist_ok=True)

In [50]:
# Copy the control file
copyfile(controlFolder / sourceFile, rootPath / domainFolder / logFolder / sourceFile);

In [51]:
# Copy this script
thisFile = 'make_folder_structure.ipynb'
copyfile(thisFile, rootPath / domainFolder / logFolder / thisFile);

In [52]:
# Get current date and time
now = datetime.now()

In [53]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + '_log.txt'
with open(rootPath / domainFolder / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Generated folder structure using ' + sourceFile]
    for txt in lines:
        file.write(txt)    